## __Faissの使い方__

- ベクトル検索ライブラリ

- Facebookが開発

- [Github](https://github.com/facebookresearch/faiss)

- CPU/GPU対応

- PythonとC++で仕様可

- 横ベクトルを使用

### __準備__

- ベクトルとして日本語単語ベクトルモデル chiVe を使用する

- [Github](https://github.com/WorksApplications/chiVe)

In [1]:
!pip install faiss-cpu | tail -n 1

In [40]:
import numpy as np
import pandas as pd
import faiss
import gensim

In [5]:
# モデルのダウンロード
!wget https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.1-mc90_gensim.tar.gz

--2023-02-05 18:58:25--  https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.1-mc90_gensim.tar.gz
Resolving sudachi.s3-ap-northeast-1.amazonaws.com (sudachi.s3-ap-northeast-1.amazonaws.com)... 52.219.136.218, 3.5.156.108, 52.219.195.34, ...
Connecting to sudachi.s3-ap-northeast-1.amazonaws.com (sudachi.s3-ap-northeast-1.amazonaws.com)|52.219.136.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545081512 (520M) [application/x-gzip]
Saving to: ‘chive-1.1-mc90_gensim.tar.gz’

chive-1.1-mc90_gens 100%[===================>] 519.83M  17.3MB/s    in 31s     

2023-02-05 18:58:57 (16.6 MB/s) - ‘chive-1.1-mc90_gensim.tar.gz’ saved [545081512/545081512]



In [6]:
# 解凍
!tar -zxvf chive-1.1-mc90_gensim.tar.gz

chive-1.1-mc90_gensim/
chive-1.1-mc90_gensim/LICENSE
chive-1.1-mc90_gensim/README.md
chive-1.1-mc90_gensim/chive-1.1-mc90.kv.vectors.npy
chive-1.1-mc90_gensim/chive-1.1-mc90.kv


In [17]:
# モデルの読み込み
model = gensim.models.KeyedVectors.load("./chive-1.1-mc90_gensim/chive-1.1-mc90.kv")

# 次元数：300次元
model["日本"].shape 

(300,)

### __ベクトルの作成__

- faissではfloat32を使用

In [32]:
# データベースベクトル
b_words = [
    "日本",
    "東京",
    "イギリス",
    "首都",
    "朝",
    "夕方",
    "夏",
    "季節",
    "東大",
    "京大",
    "受験",
    "勉強",
]

vectors = []
for word in b_words:
    vectors.append(model[word])

B = np.stack(vectors)
B.shape, B.dtype

((12, 300), dtype('float32'))

In [34]:
# クエリベクトル
q_words = [
    "アメリカ",
    "大阪",
    "昼",
    "節分",
    "大学生",
    "数学",
]

vectors = []
for word in q_words:
    vectors.append(model[word])

Q = np.stack(vectors)
Q.shape, Q.dtype

((6, 300), dtype('float32'))

### __faissによる類似度検索__

In [35]:
# ユークリッド距離に基づくインデックスを作成

index = faiss.IndexFlatL2(300)
index.add(B) # データを追加
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f315d01dd50> >

In [52]:
# 検索

# クエリと上位k件を指定
k = 3
D, I = index.search(Q, k) 

# 距離
print(D[:k])

# インデックス
print(I[:k])

[[ 5.1705456  5.7971163 12.935259 ]
 [ 7.9590526 11.888107  16.1922   ]
 [ 4.077402   5.1802454 12.870502 ]]
[[0 2 1]
 [1 0 6]
 [5 4 1]]


In [55]:
# 検索結果の表示
b_words = pd.Series(b_words)
q_words = pd.Series(q_words)

for i in range(len(Q)):
    print("クエリ：", q_words[i])
    print(f"上位{k}件：", *b_words[I[i]].values.tolist())
    print()

クエリ： アメリカ
上位3件： 日本 イギリス 東京

クエリ： 大阪
上位3件： 東京 日本 夏

クエリ： 昼
上位3件： 夕方 朝 東京

クエリ： 節分
上位3件： 季節 朝 日本

クエリ： 大学生
上位3件： 勉強 東大 京大

クエリ： 数学
上位3件： 勉強 東大 受験

